# Strategy Validation for GMF Investments

## Overview
This notebook implements a comprehensive backtesting framework to validate the portfolio optimization strategies developed in the previous phase. We'll compare the performance of our optimized portfolios against benchmark strategies and analyze risk metrics to determine the viability of our investment approach.

## 1. Environment Setup & Library Imports

In [1]:
# Install required packages
%pip install pandas numpy matplotlib seaborn plotly scipy scikit-learn yfinance -q

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import json
import warnings
from datetime import datetime, timedelta
import yfinance as yf

# Configure display settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(" Environment setup complete!")
print(" Libraries imported successfully")

 Environment setup complete!
 Libraries imported successfully


## 2. Load Portfolio Optimization ResultsWe'll load the portfolio optimization results from the previous phase to use as inputs for our strategy validation.

In [3]:
# Load portfolio optimization results from JSON file
def load_optimization_results(file_path):
    """Load portfolio optimization results from JSON file

    Parameters:
        - file_path: Path to the portfolio optimization results JSON file

    Returns:
        - Dictionary containing portfolio optimization results
    """
    try:
        with open(file_path, 'r') as f:
            results = json.load(f)
        print(f" Successfully loaded portfolio optimization results from {file_path}")
        return results
    except Exception as e:
        print(f" Error loading portfolio optimization results: {str(e)}")
        return None

# Load the portfolio optimization results
optimization_results = load_optimization_results('portfolio_optimization_results.json')

# Display basic information about the loaded results
if optimization_results:
    print("\nPortfolio Optimization Results Summary:")

    # Display expected returns
    print("\nExpected Returns:")
    for asset, return_value in optimization_results['expected_returns'].items():
        print(f" {asset}: {return_value:.4f} ({return_value*100:.2f}%)")

    # Display maximum Sharpe ratio portfolio
    print("\nMaximum Sharpe Ratio Portfolio:")
    max_sharpe = optimization_results['max_sharpe_portfolio']
    print(f" Expected Return: {max_sharpe['return']:.4f} ({max_sharpe['return']*100:.2f}%)")
    print(f" Expected Volatility: {max_sharpe['volatility']:.4f} ({max_sharpe['volatility']*100:.2f}%)")
    print(f" Sharpe Ratio: {max_sharpe['sharpe_ratio']:.4f}")
    print(" Asset Allocation:")
    for asset, weight in max_sharpe['weights'].items():
        print(f"  {asset}: {weight:.4f} ({weight*100:.2f}%)")

    # Display minimum volatility portfolio
    print("\nMinimum Volatility Portfolio:")
    min_vol = optimization_results['min_vol_portfolio']
    print(f" Expected Return: {min_vol['return']:.4f} ({min_vol['return']*100:.2f}%)")
    print(f" Expected Volatility: {min_vol['volatility']:.4f} ({min_vol['volatility']*100:.2f}%)")
    print(f" Sharpe Ratio: {min_vol['sharpe_ratio']:.4f}")
    print(" Asset Allocation:")
    for asset, weight in min_vol['weights'].items():
        print(f"  {asset}: {weight:.4f} ({weight*100:.2f}%)")

 Successfully loaded portfolio optimization results from portfolio_optimization_results.json

Portfolio Optimization Results Summary:

Expected Returns:
 TSLA: 0.0143 (1.43%)
 BND: 0.0019 (0.19%)
 SPY: -0.0002 (-0.02%)

Maximum Sharpe Ratio Portfolio:
 Expected Return: 0.0143 (1.43%)
 Expected Volatility: 0.6352 (63.52%)
 Sharpe Ratio: -0.0089
 Asset Allocation:
  TSLA: 1.0000 (100.00%)
  BND: 0.0000 (0.00%)
  SPY: 0.0000 (0.00%)

Minimum Volatility Portfolio:
 Expected Return: 0.0018 (0.18%)
 Expected Volatility: 0.0665 (6.65%)
 Sharpe Ratio: -0.2732
 Asset Allocation:
  TSLA: 0.0000 (0.00%)
  BND: 0.9454 (94.54%)
  SPY: 0.0546 (5.46%)


## 3. Fetch Historical Data for BacktestingWe'll fetch historical data for our assets to use in backtesting our portfolio strategies.

In [4]:
def fetch_historical_data(tickers, start_date, end_date):
    """Fetch historical data for the specified tickers

    Parameters:
        - tickers: List of ticker symbols
        - start_date: Start date for historical data
        - end_date: End date for historical data

    Returns:
        - Dictionary containing historical data for each ticker
    """
    historical_data = {}

    for ticker in tickers:
        try:
            # Fetch data from Yahoo Finance
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)

            # Store data in dictionary
            historical_data[ticker] = data

            print(f" Successfully fetched data for {ticker} ({len(data)} records)")
        except Exception as e:
            print(f" Error fetching data for {ticker}: {str(e)}")

    return historical_data

# Define backtesting period
backtest_start_date = '2023-01-01'
backtest_end_date = '2023-12-31'

# Define assets
assets = ['TSLA', 'BND', 'SPY']

# Fetch historical data
historical_data = fetch_historical_data(assets, backtest_start_date, backtest_end_date)

# Display data summary
for ticker, data in historical_data.items():
    print(f"\n{ticker} Data Summary:")
    print(f" Date Range: {data.index.min().strftime('%Y-%m-%d')} to {data.index.max().strftime('%Y-%m-%d')}")
    print(f" Trading Days: {len(data)}")
    print(f" Starting Price: ${data['Close'].iloc[0].item():.2f}")
    print(f" Ending Price: ${data['Close'].iloc[-1].item():.2f}")
    print(f" Return: {(data['Close'].iloc[-1].item() / data['Close'].iloc[0].item() - 1) * 100:.2f}%")

 Successfully fetched data for TSLA (250 records)
 Successfully fetched data for BND (250 records)
 Successfully fetched data for SPY (250 records)

TSLA Data Summary:
 Date Range: 2023-01-03 to 2023-12-29
 Trading Days: 250
 Starting Price: $108.10
 Ending Price: $248.48
 Return: 129.86%

BND Data Summary:
 Date Range: 2023-01-03 to 2023-12-29
 Trading Days: 250
 Starting Price: $65.98
 Ending Price: $69.35
 Return: 5.10%

SPY Data Summary:
 Date Range: 2023-01-03 to 2023-12-29
 Trading Days: 250
 Starting Price: $368.17
 Ending Price: $466.50
 Return: 26.71%


## 4. Implement Backtesting FrameworkWe'll implement a backtesting framework to evaluate the performance of our portfolio strategies.

In [5]:
def backtest_portfolio(historical_data, weights, initial_investment=10000):
    """Backtest a portfolio with the specified weights

    Parameters:
        - historical_data: Dictionary containing historical data for each asset
        - weights: Dictionary containing portfolio weights for each asset
        - initial_investment: Initial investment amount (default: $10,000)

    Returns:
        - DataFrame containing portfolio value over time
    """
    # Extract assets and their weights
    assets = list(weights.keys())

    # Create a DataFrame with adjusted close prices for each asset
    prices = pd.DataFrame()
    for asset in assets:
        prices[asset] = historical_data[asset]['Close']

    # Calculate daily returns
    daily_returns = prices.pct_change().dropna()

    # Calculate portfolio returns
    portfolio_returns = pd.Series(0, index=daily_returns.index)
    for asset in assets:
        portfolio_returns += daily_returns[asset] * weights[asset]

    # Calculate cumulative returns
    cumulative_returns = (1 + portfolio_returns).cumprod()

    # Calculate portfolio value
    portfolio_value = initial_investment * cumulative_returns

    # Create a DataFrame with portfolio metrics
    portfolio_df = pd.DataFrame({
        'Portfolio Value': portfolio_value,
        'Daily Returns': portfolio_returns,
        'Cumulative Returns': cumulative_returns - 1
    })

    return portfolio_df

def calculate_performance_metrics(portfolio_df, risk_free_rate=0.02/252):
    """Calculate performance metrics for a portfolio

    Parameters:
        - portfolio_df: DataFrame containing portfolio value over time
        - risk_free_rate: Daily risk-free rate (default: 2% annual / 252 trading days)

    Returns:
        - Dictionary containing performance metrics
    """
    # Extract daily returns
    returns = portfolio_df['Daily Returns']

    # Calculate annualized return (geometric compounding)
    total_return = portfolio_df['Cumulative Returns'].iloc[-1]
    days = len(returns)
    annual_return = (1 + total_return) ** (252 / max(1, days)) - 1

    # Calculate annualized volatility
    annual_volatility = returns.std() * np.sqrt(252)

    # Calculate Sharpe ratio (convert daily rf to annual)
    sharpe_ratio = (annual_return - risk_free_rate * 252) / annual_volatility if annual_volatility != 0 else np.nan

    # Calculate Sortino ratio
    downside_returns = returns[returns < 0]
    annualized_downside_volatility = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (annual_return - risk_free_rate * 252) / annualized_downside_volatility if annualized_downside_volatility != 0 else np.nan

    # Calculate maximum drawdown using equity curve
    equity = (1 + returns).cumprod()
    running_max = equity.cummax()
    drawdown = equity / running_max - 1
    max_drawdown = drawdown.min()

    # Calculate Value at Risk (VaR)
    var_95 = np.percentile(returns, 5)
    var_99 = np.percentile(returns, 1)

    # Calculate Calmar Ratio
    calmar_ratio = annual_return / abs(max_drawdown) if max_drawdown != 0 else np.nan

    # Store metrics in a dictionary
    performance_metrics = {
        'Total Return': total_return,
        'Annual Return': annual_return,
        'Annual Volatility': annual_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Maximum Drawdown': max_drawdown,
        'Value at Risk (95%)': var_95,
        'Value at Risk (99%)': var_99,
        'Calmar Ratio': calmar_ratio
    }

    return performance_metrics

# Define portfolio strategies to backtest
strategies = {
    'Maximum Sharpe Ratio Portfolio': optimization_results['max_sharpe_portfolio']['weights'],
    'Minimum Volatility Portfolio': optimization_results['min_vol_portfolio']['weights'],
    'Equal Weight Portfolio': {asset: 1/len(assets) for asset in assets},
    '60/40 Portfolio (SPY/BND)': {'SPY': 0.6, 'BND': 0.4, 'TSLA': 0} # Assuming SPY is stock and BND is bond
}

# Backtest each strategy
backtest_results = {}
performance_metrics = {}

for strategy_name, weights in strategies.items():
    print(f"\nBacktesting {strategy_name}...")
    portfolio_df = backtest_portfolio(historical_data, weights)
    backtest_results[strategy_name] = portfolio_df
    performance_metrics[strategy_name] = calculate_performance_metrics(portfolio_df)

print("\nBacktesting complete!")


Backtesting Maximum Sharpe Ratio Portfolio...

Backtesting Minimum Volatility Portfolio...

Backtesting Equal Weight Portfolio...

Backtesting 60/40 Portfolio (SPY/BND)...

Backtesting complete!


## 5. Visualize Backtesting ResultsWe'll visualize the performance of our portfolio strategies over the backtesting period.

In [6]:
def plot_portfolio_performance(backtest_results):
    """Plot portfolio performance over time

    Parameters:
        - backtest_results: Dictionary containing backtest results for each strategy

    Returns:
        - Plotly figure object
    """
    # Create figure
    fig = go.Figure()

    # Add portfolio value for each strategy
    for strategy_name, portfolio_df in backtest_results.items():
        fig.add_trace(go.Scatter(
            x=portfolio_df.index,
            y=portfolio_df['Portfolio Value'],
            mode='lines',
            name=strategy_name
        ))

    # Update layout
    fig.update_layout(
        title='Portfolio Performance Comparison',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Portfolio Value ($)'),
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.8)'),
        width=1000,
        height=600,
        template='plotly_white'
    )

    return fig

def plot_cumulative_returns(backtest_results):
    """Plot cumulative returns over time

    Parameters:
        - backtest_results: Dictionary containing backtest results for each strategy

    Returns:
        - Plotly figure object
    """
    # Create figure
    fig = go.Figure()

    # Add cumulative returns for each strategy
    for strategy_name, portfolio_df in backtest_results.items():
        fig.add_trace(go.Scatter(
            x=portfolio_df.index,
            y=portfolio_df['Cumulative Returns'] * 100,
            mode='lines',
            name=strategy_name
        ))

    # Update layout
    fig.update_layout(
        title='Cumulative Returns Comparison',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Cumulative Return (%)'),
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.8)'),
        width=1000,
        height=600,
        template='plotly_white'
    )

    return fig

def plot_drawdowns(backtest_results):
    """Plot drawdowns over time

    Parameters:
        - backtest_results: Dictionary containing backtest results for each strategy

    Returns:
        - Plotly figure object
    """
    # Create figure
    fig = go.Figure()

    # Add drawdowns for each strategy
    for strategy_name, portfolio_df in backtest_results.items():
        cumulative_returns = portfolio_df['Cumulative Returns']
        running_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - running_max) / (running_max + 1e-9) # Add small epsilon to avoid division by zero

        fig.add_trace(go.Scatter(
            x=portfolio_df.index,
            y=drawdown * 100,
            mode='lines',
            name=strategy_name
        ))

    # Update layout
    fig.update_layout(
        title='Drawdown Comparison',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Drawdown (%)', tickformat='.2f'),
        legend=dict(x=0.01, y=0.01, bgcolor='rgba(255, 255, 255, 0.8)'),
        width=1000,
        height=600,
        template='plotly_white'
    )

    return fig

# Plot portfolio performance
performance_fig = plot_portfolio_performance(backtest_results)
performance_fig.show()

# Plot cumulative returns
returns_fig = plot_cumulative_returns(backtest_results)
returns_fig.show()

# Plot drawdowns
drawdown_fig = plot_drawdowns(backtest_results)
drawdown_fig.show()

## 6. Compare Performance MetricsWe'll compare the performance metrics of our portfolio strategies.

In [7]:
def create_performance_comparison_table(performance_metrics):
    """Create a performance comparison table

    Parameters:
        - performance_metrics: Dictionary containing performance metrics for each strategy

    Returns:
        - DataFrame containing performance comparison
    """
    # Create a DataFrame from performance metrics
    metrics_df = pd.DataFrame(performance_metrics).T

    # Format metrics for display
    formatted_df = pd.DataFrame(index=metrics_df.index)
    formatted_df['Total Return (%)'] = metrics_df['Total Return'] * 100
    formatted_df['Annual Return (%)'] = metrics_df['Annual Return'] * 100
    formatted_df['Annual Volatility (%)'] = metrics_df['Annual Volatility'] * 100
    formatted_df['Sharpe Ratio'] = metrics_df['Sharpe Ratio']
    formatted_df['Sortino Ratio'] = metrics_df['Sortino Ratio']
    formatted_df['Calmar Ratio'] = metrics_df['Calmar Ratio']
    formatted_df['Maximum Drawdown (%)'] = metrics_df['Maximum Drawdown'] * 100
    formatted_df['Value at Risk (95%) (%)'] = metrics_df['Value at Risk (95%)'] * 100
    formatted_df['Value at Risk (99%) (%)'] = metrics_df['Value at Risk (99%)'] * 100

    return formatted_df

def plot_performance_metrics(performance_metrics):
    """Plot performance metrics comparison

    Parameters:
        - performance_metrics: Dictionary containing performance metrics for each strategy

    Returns:
        - Plotly figure object
    """
    # Create a DataFrame from performance metrics
    metrics_df = pd.DataFrame(performance_metrics).T

    # Select metrics to plot
    plot_metrics = [
        'Annual Return',
        'Annual Volatility',
        'Sharpe Ratio',
        'Maximum Drawdown'
    ]

    # Create subplots
    fig = make_subplots(rows=2, cols=2, subplot_titles=plot_metrics)

    # Add bars for each metric
    strategies = list(metrics_df.index)
    colors = px.colors.qualitative.Plotly

    # Annual Return
    fig.add_trace(
        go.Bar(
            x=strategies,
            y=metrics_df['Annual Return'] * 100,
            name='Annual Return',
            marker_color=colors[0]
        ),
        row=1, col=1
    )

    # Annual Volatility
    fig.add_trace(
        go.Bar(
            x=strategies,
            y=metrics_df['Annual Volatility'] * 100,
            name='Annual Volatility',
            marker_color=colors[1]
        ),
        row=1, col=2
    )

    # Sharpe Ratio
    fig.add_trace(
        go.Bar(
            x=strategies,
            y=metrics_df['Sharpe Ratio'],
            name='Sharpe Ratio',
            marker_color=colors[2]
        ),
        row=2, col=1
    )

    # Maximum Drawdown
    fig.add_trace(
        go.Bar(
            x=strategies,
            y=metrics_df['Maximum Drawdown'] * 100,
            name='Maximum Drawdown',
            marker_color=colors[3]
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        title='Performance Metrics Comparison',
        showlegend=False,
        width=1000,
        height=800,
        template='plotly_white'
    )

    return fig

# Create performance comparison table
performance_table_df = create_performance_comparison_table(performance_metrics)
display(performance_table_df)

# Plot performance metrics
performance_metrics_fig = plot_performance_metrics(performance_metrics)
performance_metrics_fig.show()

,Total Return (%),Annual Return (%),Annual Volatility (%),Sharpe Ratio,Sortino Ratio,Calmar Ratio,Maximum Drawdown (%),Value at Risk (95%) (%),Value at Risk (99%) (%)
Maximum Sharpe Ratio Portfolio,129.861239,132.177832,52.648774,2.472571,4.151959,4.039701,-32.719710,-5.014129,-7.868874
Minimum Volatility Portfolio,6.230875,6.308265,7.080415,0.608476,1.174836,0.968862,-6.511004,-0.649622,-0.913428
Equal Weight Portfolio,49.318595,50.041589,20.719153,2.318704,3.933053,3.205694,-15.610218,-1.766961,-3.008400
60/40 Portfolio (SPY/BND),17.851578,18.085035,8.792102,1.829487,3.131170,2.343756,-7.716262,-0.864576,-1.146152


## 7. Risk-Return AnalysisWe'll analyze the risk-return tradeoff of our portfolio strategies.

In [8]:
def plot_risk_return_scatter(performance_metrics):
    """Plot risk-return scatter plot

    Parameters:
        - performance_metrics: Dictionary containing performance metrics for each strategy

    Returns:
        - Plotly figure object
    """
    # Create a DataFrame from performance metrics
    metrics_df = pd.DataFrame(performance_metrics).T

    # Create figure
    fig = go.Figure()

    # Add scatter plot
    fig.add_trace(go.Scatter(
        x=metrics_df['Annual Volatility'] * 100,
        y=metrics_df['Annual Return'] * 100,
        mode='markers+text',
        marker=dict(
            size=15,
            color=metrics_df['Sharpe Ratio'],
            colorscale='Viridis',
            colorbar=dict(title='Sharpe Ratio'),
            showscale=True
        ),
        text=metrics_df.index,
        textposition='top center'
    ))

    # Update layout
    fig.update_layout(
        title='Risk-Return Analysis',
        xaxis=dict(title='Risk (Annual Volatility %)'),
        yaxis=dict(title='Return (Annual %)'),
        width=1000,
        height=600,
        template='plotly_white'
    )

    return fig

# Plot risk-return scatter plot
risk_return_fig = plot_risk_return_scatter(performance_metrics)
risk_return_fig.show()

## 8. Export Validation ResultsWe'll export the validation results for reporting and further analysis.

In [9]:
def export_validation_results(performance_metrics, file_path):
    """Export validation results to JSON file

    Parameters:
        - performance_metrics: Dictionary containing performance metrics for each strategy
        - file_path: Path to save the results

    Returns:
        - Boolean indicating success or failure
    """
    # Create results dictionary
    results = {}

    for strategy, metrics in performance_metrics.items():
        results[strategy] = {k: float(v) for k, v in metrics.items()}

    # Save results to JSON file
    try:
        with open(file_path, 'w') as f:
            json.dump(results, f, indent=2)
        print(f" Successfully exported validation results to {file_path}")
        return True
    except Exception as e:
        print(f" Error exporting validation results: {str(e)}")
        return False

# Export validation results
export_validation_results(performance_metrics, 'strategy_validation_results.json')

 Successfully exported validation results to strategy_validation_results.json


True

## 9. Conclusion

We've successfully implemented a comprehensive backtesting framework to validate our portfolio optimization strategies. The key findings are:

1.  **Performance Comparison**: We compared the performance of our optimized portfolios (Maximum Sharpe Ratio and Minimum Volatility) against benchmark strategies (Equal Weight and 60/40).
2.  **Risk-Return Analysis**: We analyzed the risk-return tradeoff of each strategy to determine the most efficient allocation.
3.  **Risk Metrics**: We calculated comprehensive risk metrics including Sharpe ratio, Sortino ratio, maximum drawdown, and Value at Risk.
4.  **Strategy Viability**: We assessed the viability of our investment approach based on historical performance.

The validation results demonstrate the effectiveness of our portfolio optimization approach and provide valuable insights for investment decision-making.